# Segmenting and Clustering Neighborhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.


## Section 1: Create a dataframe of Toronto postal code

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 
3. To create the above dataframe:


* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.


In [1]:
#!pip install beautifulsoup4 # Install package for web scrapping
#!pip install lxml #Install parcer? for 
#!pip install html5lib

from bs4 import BeautifulSoup
import requests
import pandas as pd
source = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.').text
soup = BeautifulSoup(source,'lxml')
obj=soup.find('tr')
#print(obj.prettify())
data=pd.DataFrame(columns=['Postal Code', 'Borough','Neighborhood']) #create blank data frame for scrapping
list=[]
for obj in soup.find_all('tr'):
    list=[]
    try:
        for tr in obj.find_all('td'):
            #print(tr.text)
            list=list+[tr.text]
    except Exception as e:
        list=[]
    if len(list)==3:
        #print(list)
        data=data.append({'Postal Code': list[0], 'Borough': list[1],'Neighborhood': list[2]},ignore_index=True)
        #print(data)
#data.head()
print('Total number of M-postal code:',data.shape[0])
df=data[data.Borough != 'Not assigned']
df['Neighborhood']= df['Neighborhood'].str.replace('\n', '')
print('Total number of assigned M-postal code:',df.shape[0])
df.head()

Total number of M-postal code: 287
Total number of assigned M-postal code: 210


/Users/lbpham/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [12]:
print('The number of rows of the dataframe: ',df.shape[0])

The number of rows of the dataframe:  210


## Section 2: Adding geo-coordinates to the dataframe

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Important Note: There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (2 marks)

In [3]:
#!pip install geocoder
import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None
# loop until you get the coordinates
#while(lat_lng_coords is None):
#g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
#lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]
#print(lat_lng_coords)

url='https://cocl.us/Geospatial_data'
geo = pd.read_csv(url)
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [4]:
final=pd.merge(df, geo, on='Postal Code')
final=final.sort_values(by=['Postal Code']).reset_index(drop=True)
final.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Port Union,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Highland Creek,43.784535,-79.160497


## Section 3: Explore and cluster the neighborhoods in Toronto


In [5]:
#Import all the libraries needed
#!pip install geopy
#!pip install folium

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [6]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [7]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for i in range(final.shape[0]):
    neighborhood=final.loc[i,'Neighborhood']
    borough=final.loc[i,'Borough']
    lat=final.loc[i,'Latitude']
    lng=final.loc[i,'Longitude']
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [8]:
toronto=final[final['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,Riverdale,43.679557,-79.352188
2,M4K,East Toronto,The Danforth West,43.679557,-79.352188
3,M4L,East Toronto,The Beaches West,43.668999,-79.315572
4,M4L,East Toronto,India Bazaar,43.668999,-79.315572


In [9]:
import matplotlib.pyplot as plt
import random

area=toronto.Borough.unique()
color={}

for i in range(len(area)):
    #print(area[i])
    color[area[i]]="#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
    #print(color[area[i]])
color                                

{'East Toronto': '#C72393',
 'Central Toronto': '#50E990',
 'Downtown Toronto': '#C083FB',
 'West Toronto': '#0239DF'}

In [10]:
# create map of Manhattan using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers to map
for i in range(toronto.shape[0]):
    label='N: ' + toronto.loc[i,'Neighborhood'] + ' | ' + 'B: '+toronto.loc[i,'Borough']
    lat=toronto.loc[i,'Latitude']
    lng=toronto.loc[i,'Longitude']
    cluster=toronto.loc[i,'Borough']
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color[toronto.loc[i,'Borough']],
        fill=True,
        fill_color=color[toronto.loc[i,'Borough']],
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  

### Map of Toronto with Neighborhood clustered by Borough

In [11]:
map_downtown